In [1]:
import os
import pandas as pd

In [6]:
def coco_form():
    coco_format = dict(
        info=dict(
            year=2021, version="1.0", description="Pedestrian", contributor="AI Hub", url="https://aihub.or.kr/aidata/136", date_created='2021-12-16 12:49:00'
        ),
        licenses=[
            dict(id=0, name='', url='')
        ],
        categories=[
            dict(id=0, name='person', supercategory=None),
            dict(id=1, name='stroller', supercategory=None),
            dict(id=2, name='bicycle', supercategory=None),
            dict(id=3, name='wheelchair', supercategory=None)
        ],
        images=[
        ],
        annotations=[
        ]
    )

    # with open('coco_format.json','w') as f:
    #     json.dump(coco_format, f, indent=4)

    return coco_format

In [7]:
coco_form()

{'info': {'year': 2021,
  'version': '1.0',
  'description': 'Pedestrian',
  'contributor': 'AI Hub',
  'url': 'https://aihub.or.kr/aidata/136',
  'date_created': '2021-12-16 12:49:00'},
 'licenses': [{'id': 0, 'name': '', 'url': ''}],
 'categories': [{'id': 0, 'name': 'person', 'supercategory': None},
  {'id': 1, 'name': 'stroller', 'supercategory': None},
  {'id': 2, 'name': 'bicycle', 'supercategory': None},
  {'id': 3, 'name': 'wheelchair', 'supercategory': None}],
 'images': [],
 'annotations': []}

In [10]:
# file in list
# https://jaeyung1001.tistory.com/44
from ast import literal_eval
all_df = pd.read_csv(os.path.join('/Users/sm/Downloads','detection_train.csv'), converters={'box':literal_eval})
all_df

,root_folder,sub_folder,xml_name,file_name,occluded,classes,box,class_no
0,Bbox_16_new,Bbox_0969,1007_22.xml,MP_SEL_MIN_000001.jpg,0,person,"[739.96, 452.13, 775.20, 540.70]",0
1,Bbox_16_new,Bbox_0969,1007_22.xml,MP_SEL_MIN_000001.jpg,0,person,"[794.29, 447.59, 832.80, 541.70]",0
2,Bbox_16_new,Bbox_0969,1007_22.xml,MP_SEL_MIN_000003.jpg,0,wheelchair,"[1088.50, 463.35, 1154.58, 574.70]",3
3,Bbox_16_new,Bbox_0969,1007_22.xml,MP_SEL_MIN_000003.jpg,1,bicycle,"[714.49, 505.25, 894.90, 698.50]",2
4,Bbox_16_new,Bbox_0969,1007_22.xml,MP_SEL_MIN_000003.jpg,1,person,"[907.08, 413.30, 931.63, 480.82]",0
...,...,...,...,...,...,...,...,...
28184,Bbox_18_new,Bbox_1022,1031_09.xml,MP_SEL_MIN_013601.jpg,0,person,"[1114.70, 234.30, 1333.50, 935.60]",0
28185,Bbox_18_new,Bbox_1022,1031_09.xml,MP_SEL_MIN_013601.jpg,1,person,"[750.67, 244.47, 771.40, 323.70]",0
28186,Bbox_18_new,Bbox_1022,1031_09.xml,MP_SEL_MIN_013601.jpg,1,person,"[764.63, 248.88, 781.63, 320.97]",0
28187,Bbox_18_new,Bbox_1022,1031_09.xml,MP_SEL_MIN_013602.jpg,0,person,"[576.00, 189.70, 601.80, 238.80]",0


In [12]:
# define fields
# images
LICENCE = 0
URL = None
DATA_CAPTURED = None
#BBOX
WIDTH=1920
HEIGHT=1080
SEGMENTATION = []
ISCROWD = 0

file_list = all_df["file_name"].tolist()
set_file_list = set(file_list)
print(f'total #bbox : {len(file_list)},\timages: {len(set_file_list)}')

total #bbox : 28189,	images: 6913


In [13]:
from tqdm import tqdm
image_id_index = 0
coco = coco_form()
for image in tqdm(set_file_list):
    condition = all_df['file_name'] == image
    coco['images'].append(
        dict(license=LICENCE, url=URL, file_name=image, height=HEIGHT, width=WIDTH, data_captured=DATA_CAPTURED, id=image_id_index)
        )

    box_id_index = 0
    for bbox in all_df[condition].to_dict('records'):
        bbox['box'] = [float(idx) for idx in bbox['box']]
        #width
        bbox['box'][2] = bbox['box'][2] - bbox['box'][0] 
        #height
        bbox['box'][3] = bbox['box'][3] - bbox['box'][1]
        coco['annotations'].append(
            dict(id=box_id_index, image_id=image_id_index, category_id=bbox['class_no'], bbox=bbox['box'], 
                area=round(bbox['box'][2] * bbox['box'][3], 3), segmentation=SEGMENTATION, iscrowd=ISCROWD)
            )
        box_id_index+=1
    image_id_index+=1

100%|██████████| 6913/6913 [00:18<00:00, 374.81it/s]


In [14]:
# 잘 들어갔는지 검증
print(f'total #bbox : {len(coco["annotations"])},\timages: {len(coco["images"])}')

total #bbox : 28189,	images: 6913


In [15]:
import json
with open('pedestrian_train.json', 'w') as f:
    json.dump(coco,f, indent=4)